# Capado Test Notes

## Abstract

KM Snapshots deiver a significant startup performance boot for Capado's Spring Boot and MicroNaut based applications.

The following table shows the time for program startup, as measured by how long it takes before the application responds to HTTP GET requests, for the native applications and for KM snapshots of the applications.

Program | Native | KM Snapshot | Speedup
------- | -------| ------- | ----
Spring Boot | 2.907 | 0.072 | 40x
Micronaut | 1.352 | 0.074 | 18x


## Details

Startup time for applications is becoming increasingly important as the world moves from monolitic applications to applications as a collection of microservces. As the number of microservices grows it becomes increasingly expensive to maintain acceptable microservice response time by pre-running binaries for these microservices in memory.

Many microservices are in fact implenented in Java using frameworks like Spring Boot and Micronaut. These frameworks automate configuration and compoment linkage tasks that are extrememly error-prone when done by hand. The cost for this is long start-up times as the compoments are initialized and the application is ready to process requests.

The Konatin Monitor (KM) can start an on-disk image of an application after it has been initialized to be started and ready to process requests in a fraction of the time of the original application.

These tests were run against the Spring Boot application at `https://github.com/CopadoSolutions/springboot-poc.git`.

The application requires a `redis` server and a `postgres` server. Docker containers with the latest public image for these components were used.

Time to application ready was measured by starting a client application that attempts to make a HTTP `GET` command. When the `GET` succeeds, the current time in nanoseconds is printed. The time in nanoseconds is printed when the application is started. The value `successful_get_time - application_start_time` is the startup time.

Two scenarios are compared:
1. The application started with native java
2. A KM snapshot of the application is started.

Each scenario was run five times. The mean and standard deviation are calculated for each scenario.

## Spring Boot Results

The mean startup time for the native Spring Boot Java application was 2.907 seconds with a standard deviation of 0.039 seconds. In contrast, a KM snapshot of the application had a mean startup time of 0.072 seconds with a 0.010 second standard deviation.

This represents a 40x improvement over native Spring Boot.

## MicroNaut Results

The mean startup time for the native Micrinaut application was 1.352 seconds with a standard deviation of 0.004 seconds. In contrast, a KM snapshot of the application had a mean startup time of 0.074 seconds with a standard deviation of 0.004 seconds.

This represents a 18x improvement over native Micronaut.

## Background

Python code to generate results.


In [32]:
# Common imports and function
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def mean_time(times):
    # time is nanosecond value. Convert to seconds.
    return (np.mean([(x[1] - x[0]) for x in times])/1000000000, 
            np.std([(x[1] - x[0]) for x in times])/1000000000)

### Sprint Boot Native
Native runs `java -jar springboot-poc-0.0.1-SNAPSHOT.jar`.

In [79]:
# [start, ready] timestamps for native java (No Docker)
native = [[1594330616955415687, 1594330619922285498],
          [1594330700336089732, 1594330703268789829],
          [1594330881783208703, 1594330884639513602],
          [1594330938616865576, 1594330941514508415],
          [1594330985850950981, 1594330988732767528]]

native_mean = mean_time(native)
print("native mean: {} stddev: {}".format(native_mean[0], native_mean[1]))

mean: 2.9070668386 stddev: 0.038824278358718295


### Spring Boot KM Snapshot

Snapshot runs: `km --resume <snapshot>`.

In [86]:
# KM Snapshot start (Inside Docker for file system)
km_snap = [
            [1594346248289880216, 1594346248363207645],
            [1594346340914741204, 1594346341004931036],
            [1594346415514450172, 1594346415588229106],
            [1594346451486340815, 1594346451549087212],
            [1594346518005153220, 1594346518068040115]
          ]

kmsnap_mean = mean_time(km_snap)
print("km snapshot mean: {} stddev: {}".format(kmsnap_mean[0], kmsnap_mean[1]))
print("snapshot speedup: {}".format(native_mean[0] / kmsnap_mean[0]))

km snapshot mean: 0.0725858974 stddev: 0.010027543413225606
snapshot speedup: 40.05002270041508


### Micronaut Native


In [102]:
nm_native = [
    [1594355798575781943, 1594355799917436852],
    [1594356008536292824, 1594356009919872323],
    [1594356090868155823, 1594356092212039880],
    [1594356148473063658, 1594356149815613916],
    [1594356254815930466, 1594356256165408988]
]
mn_native_mean = mean_time(nm_native)
#[n[1] - n[0] for n in nm_native]
print("micronaut native mean: {} stddev: {}".format(mn_native_mean[0], mn_native_mean[1]))

micronaut native mean: 1.352229449 stddev: 0.01590940876727714


### Micronaut KM Snapshot

In [105]:
mn_kmsnap = [
    [1594589028637859804, 1594589028709518230],
    [1594589231424698394, 1594589231506652927],
    [1594589277646541020, 1594589277719343920],
    [1594589343740795895, 1594589343815068462],
    [1594589389651950692, 1594589389724188317],
]

mn_kmsnap_mean = mean_time(nm_kmsnap)
print("km snapshot mean: {} stddev: {}".format(mn_kmsnap_mean[0], mn_kmsnap_mean[1]))
print("snapshot speedup: {}".format(mn_native_mean[0] / mn_kmsnap_mean[0]))

km snapshot mean: 0.0745852102 stddev: 0.0037857236025562882
snapshot speedup: 18.129994477108813


# Snapshot Creation

Time creation of snapshot. KM process exit counted as snapshot fully created.

In [113]:
# Removed all coredump printing
kmsnap_create_silent = [
    [1594607583125984451, 1594607583565739634],
    [1594607633453041740, 1594607633887310192],
    [1594607703715755381, 1594607704139619280],
]

mean_time(kmsnap_create_silent)

(0.432629178, 0.00659032832907289)